In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
!pip install transformers
!pip install tensorflow


In [ ]:

from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas as pd
import numpy as np
import os

In [ ]:
# Load and preprocess dataset
df = pd.read_csv("bbc-text.csv")

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['category'])
num_classes = df['label'].nunique()
labels_onehot = to_categorical(df['label'], num_classes=num_classes)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), labels_onehot, test_size=0.2, random_state=42
)

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
# Tokenization
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='tf')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, return_tensors='tf')

def make_dataset(encodings, labels, batch_size):
    return tf.data.Dataset.from_tensor_slices((
        dict(encodings),
        labels
    )).shuffle(1000).batch(batch_size)


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [ ]:
# Install dependencies (if not already done)
!pip install transformers
!pip install tensorflow
!pip install pandas scikit-learn

# Imports
import os
import pandas as pd
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizer
from transformers.optimization_tf import create_optimizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load and preprocess data
df = pd.read_csv("bbc-text.csv")

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['category'])
num_classes = df['label'].nunique()
labels = df['label'].values  # integer labels (NOT one-hot)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), labels, test_size=0.2, random_state=42
)

# Tokenize inputs
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
MAX_LEN = 256

train_encodings = tokenizer(train_texts, truncation=True, padding='max_length',
                            max_length=MAX_LEN, return_tensors='tf')
val_encodings = tokenizer(val_texts, truncation=True, padding='max_length',
                          max_length=MAX_LEN, return_tensors='tf')

# Dataset creator
def make_dataset(encodings, labels, batch_size):
    return tf.data.Dataset.from_tensor_slices((
        dict(encodings),
        labels
    )).shuffle(1000).batch(batch_size)

# Grid Search
best_accuracy = 0
best_model = None
best_params = {}
results = []

learning_rates = [5e-5, 3e-5, 2e-5]
batch_sizes = [8, 16]  # Safe for Colab
epochs_list = [3, 4]

for lr in learning_rates:
    for bs in batch_sizes:
        for ep in epochs_list:
            print(f"\nTraining with lr={lr}, batch_size={bs}, epochs={ep}")

            train_dataset = make_dataset(train_encodings, train_labels, bs)
            val_dataset = make_dataset(val_encodings, val_labels, bs)

            model = TFDistilBertForSequenceClassification.from_pretrained(
                'distilbert-base-uncased', num_labels=num_classes
            )

            steps_per_epoch = len(train_dataset)
            num_train_steps = steps_per_epoch * ep

            optimizer, _ = create_optimizer(
                init_lr=lr,
                num_train_steps=num_train_steps,
                num_warmup_steps=0
            )

            model.compile(
                optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
            )

            history = model.fit(train_dataset, validation_data=val_dataset, epochs=ep, verbose=1)

            val_acc = history.history['val_accuracy'][-1]
            val_loss = history.history['val_loss'][-1]
            print(f"→ val_accuracy: {val_acc:.4f}, val_loss: {val_loss:.4f}")

            results.append({
                'learning_rate': lr,
                'batch_size': bs,
                'epochs': ep,
                'val_accuracy': val_acc,
                'val_loss': val_loss
            })

            if val_acc > best_accuracy:
                best_accuracy = val_acc
                best_model = model
                best_params = {'learning_rate': lr, 'batch_size': bs, 'epochs': ep}

# Save results
results_df = pd.DataFrame(results)
results_df.to_csv("grid_search_results.csv", index=False)

# Save best model
save_path = "./saved_models/best_bbc_model"
os.makedirs(save_path, exist_ok=True)
best_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("\n✅ Best hyperparameters:", best_params)
print("🎯 Best validation accuracy:", round(best_accuracy, 4))



Training with lr=5e-05, batch_size=8, epochs=3


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
223/223 [==============================] - 75s 245ms/step - loss: 1.6682 - accuracy: 0.2096 - val_loss: 1.6094 - val_accuracy: 0.1730
Epoch 2/3
223/223 [==============================] - 52s 235ms/step - loss: 1.6094 - accuracy: 0.2180 - val_loss: 1.6094 - val_accuracy: 0.1730
Epoch 3/3
223/223 [==============================] - 51s 230ms/step - loss: 1.6094 - accuracy: 0.2230 - val_loss: 1.6094 - val_accuracy: 0.1730
→ val_accuracy: 0.1730, val_loss: 1.6094

Training with lr=5e-05, batch_size=8, epochs=4


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/4
223/223 [==============================] - 76s 244ms/step - loss: 1.6964 - accuracy: 0.2090 - val_loss: 1.6094 - val_accuracy: 0.2090
Epoch 2/4
223/223 [==============================] - 51s 230ms/step - loss: 1.6094 - accuracy: 0.1933 - val_loss: 1.6094 - val_accuracy: 0.2090
Epoch 3/4
223/223 [==============================] - 52s 235ms/step - loss: 1.6094 - accuracy: 0.1966 - val_loss: 1.6094 - val_accuracy: 0.2090
Epoch 4/4
223/223 [==============================] - 52s 233ms/step - loss: 1.6094 - accuracy: 0.1966 - val_loss: 1.6094 - val_accuracy: 0.2090
→ val_accuracy: 0.2090, val_loss: 1.6094

Training with lr=5e-05, batch_size=16, epochs=3


...

Epoch 1/4
223/223 [==============================] - 78s 251ms/step - loss: 2.0591 - accuracy: 0.2629 - val_loss: 1.6094 - val_accuracy: 0.2697
Epoch 2/4
223/223 [==============================] - 51s 230ms/step - loss: 1.6156 - accuracy: 0.2708 - val_loss: 1.6094 - val_accuracy: 0.2854
Epoch 3/4
223/223 [==============================] - 51s 231ms/step - loss: 1.6151 - accuracy: 0.2815 - val_loss: 1.6094 - val_accuracy: 0.3124
Epoch 4/4
223/223 [==============================] - 53s 237ms/step - loss: 1.6076 - accuracy: 0.2888 - val_loss: 1.6094 - val_accuracy: 0.3124
→ val_accuracy: 0.3124, val_loss: 1.6094

Training with lr=2e-05, batch_size=16, epochs=3


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
112/112 [==============================] - 74s 463ms/step - loss: 2.1034 - accuracy: 0.2809 - val_loss: 1.6094 - val_accuracy: 0.2562
Epoch 2/3
112/112 [==============================] - 50s 443ms/step - loss: 1.6070 - accuracy: 0.2725 - val_loss: 1.6094 - val_accuracy: 0.3079
Epoch 3/3
112/112 [==============================] - 49s 438ms/step - loss: 1.6067 - accuracy: 0.3230 - val_loss: 1.6094 - val_accuracy: 0.3079
→ val_accuracy: 0.3079, val_loss: 1.6094

Training with lr=2e-05, batch_size=16, epochs=4


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/4
112/112 [==============================] - 76s 462ms/step - loss: 1.8516 - accuracy: 0.2927 - val_loss: 1.6094 - val_accuracy: 0.2517
Epoch 2/4
112/112 [==============================] - 49s 437ms/step - loss: 1.6085 - accuracy: 0.2871 - val_loss: 1.6094 - val_accuracy: 0.2494
Epoch 3/4
112/112 [==============================] - 49s 441ms/step - loss: 1.6094 - accuracy: 0.2933 - val_loss: 1.6094 - val_accuracy: 0.2494
Epoch 4/4
112/112 [==============================] - 49s 439ms/step - loss: 1.6159 - accuracy: 0.2910 - val_loss: 1.6094 - val_accuracy: 0.2899
→ val_accuracy: 0.2899, val_loss: 1.6094

✅ Best hyperparameters: {'learning_rate': 2e-05, 'batch_size': 8, 'epochs': 3}
🎯 Best validation accuracy: 0.8697


In [ ]:
from google.colab import files
files.download("grid_search_results.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>